# Librería para procesamiento de datos PyEO

Adaptación realizada por:

- David Alonso Rueda Rodríguez - Ing. Catasral y Geodesta
- Luís Manuel Guerrero Buelvas - Ing. Ambiental

## Práctica 01. Prueba Inicialización de datos

En este caso se verificarán las rutas de instalación de los servicios requeridos:

### 0. Requerimientos para ejecución de la librería

Este paquete de datos requiere la previa instalación de los siguientes elementos:

- Anaconda
- Descarga y descompresión del paquete Sen2Core (http://step.esa.int/main/third-party-plugins-2/sen2cor/)
- Descarga del repositorio de datos git del proyecto PyEO (https://github.com/clcr/pyeo)
- Libreŕia Folium (https://python-visualization.github.io/folium/installing.html)

### 1. Importando las librerías

In [3]:
import sys                                                                        # Importar librería del sistema
sys.path.append("/home/davo/Documentos/pyeo")                                     # Agregar la ruta en donde está alojada la pyeo
sen2cor_path = r"/home/davo/Documentos/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"  # Agregar la ruta del programa Sen2Core, programa original de la ESA
import pyeo.core as pyeo                                                          # Importar la librería PyEO para análisis de Imágenes Sentinel.
import matplotlib.pyplot as plt                                                   # Importar librería matplotlib para presentación de gráficos
import gdal                                                                       # Importar librería gdal para trabajo con datos espaciales
import pprint                                                                     # Importar librería para impresión de datos al sistema
import os                                                                         # Importar librería os para manejo de archivos

### 2. Creación de archivo log

El objetivo de esta actividad es actividad el archvio para seguimiento de las actividades realizadas

In [4]:
# Creación de archivo para seguimiento de actividades
pyeo.init_log("logs/training_log.log")

2019-05-15 21:41:55,247: INFO: ****PROCESSING START****


<Logger pyeo.core (DEBUG)>

### 3. Comprobación de las funciones

Esta prueba llama a la función **is_tif** que hace parte de la librería pyeo, la función únicamente comprueba que el archivo tenga la extensión .tif, en caso positivo retorna el valor **True** en caso contrario **False**.

In [8]:
# Esta prueba permite identificar si el llamado a las funciones está trabajando correctamento
pyeo.is_tif("prueba.tif")

True

In [5]:
def show_satellite_image(image_path):
    img = gdal.Open(image_path)
    array = img.GetVirtualMemArray()
    img_view = array.transpose([1,2,0])
    plt.imshow(img_view)
    array = None
    img = None

In [77]:
def mapa_area_interes(json_path, name_area, zoom_op, tipo_image = 'Stamen Terrain'):
    """Función genera mapa de interés, leyendo un archivo json y estimando el centroide para agregar una marca de posición con el nombre"""
    import folium as fl
    import os
    
    area_interes = os.path.join('',json_path)    
    m= fl.Map( 
        location=[4.464769,-75.507265],
        tiles=tipo_image,
        zoom_start = zoom_op,
    )
    fl.GeoJson(
        area_interes,
        name=name_area
    ).add_to(m)
    fl.Marker(
        location= [4.464769,-75.507265],
        popup = name_area
    ).add_to(m)
    fl.LayerControl().add_to(m)
    print("Mapa del {}" .format(name_area))
    return m

### 4. Obtener datos de acceso a programa Copernicus

Se leerán los datos de usuario almacenados en un archivo **.ini**, para confirmar que todo ha funcionado se imprimirá el nombre del usuario en el archivo

In [14]:
import configparser                                            # Importar módulo para acceso a archivo de configuración
conf = configparser.ConfigParser()                             # Crear un objeto ConfigParser y asignarlo a la variables conf
archivo_configuracion = "../lic/kinangop.ini"                  # Nombre y ruta del archivo con las variables de acceso al servicio Copernicus
conf.read(archivo_configuracion)                               # Leer el archivo con las variables de acceso al servicio Copernicus
print('Nmbre de usuario: {}' .format(conf['sent_2']['user']))  # Mostrar los datos de la configuración

Nmbre de usuario: polybispo


### 5. Selección y descarga de datos
#### 5.1 Identificación del área

El área de trabajo para consulta se localiza en Colombia, en el Departamento del Tolima en el municipio de Cajamarca. En el área se presenta una zona con pinos destinados al aprovechamiento forestal. La siguiente ventana presenta el área de interés:




In [79]:
nombre_mapa = "Área de Interés"
mapa_area_interes ('src/AOI.json', nombre_mapa, 12)

Mapa del Área de Interés
